In [2]:
import plotly.express as px
import pandas as pd
import datasets
from main import filter_out_wrong_data
from named_entity.named_entity_model import NamedEntityModel

In [3]:
pd.set_option('display.max_rows', 10000)

In [4]:
train_df=pd.read_csv('merged_train.tsv',sep='\t')
test_df=pd.read_csv('merged_test.tsv',sep='\t')

# train_df = filter_out_wrong_data(train_df)
# test_df = filter_out_wrong_data(test_df)
# ner_model = NamedEntityModel(model_path='models/ner_size_quality_tests_0.6/')

C:\Users\micha\AppData\Local\Temp\ipykernel_9900\1698846674.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df=pd.read_csv('merged_train.tsv',sep='\t')


In [6]:
def add_problem(df, description, pattern, problems, contains=True):
    if contains:
        df['is_problem']=df['text'].str.contains(pattern)
    else:
        df['is_problem']=~df['text'].str.contains(pattern)
    problem_df=df[df['is_problem']==True]
    problem={"description": description,'row_count': {len(problem_df)}}
    print(f"Problem: {description}. Number of examples affected: {len(problem_df)} out of {len(df)} ({round(100*len(problem_df)/len(df),2)}%)")
    problems.append(problem)

In [7]:
df=pd.concat([train_df,test_df])

In [8]:
problems=[]
add_problem(df=df,description="Swapped two entities",pattern=r".*<.?e2>.*<.?e1>.*", problems=problems)
add_problem(df=df,description="Empty entity in text",pattern=r'(.*<e1></e1>.*)|(.*<e2></e2>.*)', problems=problems)
add_problem(df=df,description="Multiple entities in text",pattern=r"(.*<e1>.*<e1>.*)|(.*<e2>.*<e2>.*)",problems=problems)
add_problem(df=df,description="Does not contain correct entity tag pattern (<e1>,</e1>,<e2>,</e2>)",
            pattern=r".*<e1>.*</e1>.*<e2>.*</e2>.*",problems=problems,contains=False)
add_problem(df=df,description="The entity consists only of a part of a word",
            pattern=r"(.*[^ ]<e1>.*)|(.*</e1>[^ ].*)|(.*[^ ]<e2>.*)|(.*</e2>[^ ].*)",problems=problems)
add_problem(df=df,description="&nbsp in text",pattern=r".*&.?nbsp.*",problems=problems)
add_problem(df=df,description="Contains infobox writer", pattern=r".*infobox writer.*",problems=problems)
add_problem(df=df,description="Contains Koordinaten", pattern=r".*Koordinaten.*",problems=problems)
add_problem(df=df,description="Contains Coordinates", pattern=r".*Coordinates: .*",problems=problems)

Problem: Swapped two entities. Number of swapped entities: 37901 out of 1377745 examples (2.75%)


C:\Users\micha\AppData\Local\Temp\ipykernel_9900\3718452000.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['is_problem']=df['text'].str.contains(pattern)


Problem: Empty entity in text. Number of swapped entities: 92 out of 1377745 examples (0.01%)
Problem: Multiple entities in text. Number of swapped entities: 2 out of 1377745 examples (0.0%)
Problem: Does not contain correct entity tag pattern (<e1>,</e1>,<e2>,</e2>). Number of swapped entities: 48212 out of 1377745 examples (3.5%)
Problem: The entity consists only of a part of a word. Number of swapped entities: 799950 out of 1377745 examples (58.06%)
Problem: &nbsp in text. Number of swapped entities: 5903 out of 1377745 examples (0.43%)
Problem: Contains infobox writer. Number of swapped entities: 13 out of 1377745 examples (0.0%)
Problem: Contains Koordinaten. Number of swapped entities: 2590 out of 1377745 examples (0.19%)
Problem: Contains Coordinates. Number of swapped entities: 2281 out of 1377745 examples (0.17%)


In [10]:
ner_model=NamedEntityModel(model_path)

In [11]:
# original_df,predictions_df=ner_model.add_predictions_and_correctness_label_to_dataframe(test_df)

OSError: Can't load the configuration of './ner'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './ner' is the correct path to a directory containing a config.json file

In [16]:
relation_counts = df['label'].value_counts()
relation_percentages = round(relation_counts / len(df) * 100,2)
relation_share_df = pd.DataFrame({'Count': relation_counts, 'Percentage': relation_percentages})
display(relation_share_df)

,Count,Percentage
is-where,213255,15.48
birth-place,181345,13.16
has-type,170311,12.36
movie-has-director,150585,10.93
has-occupation,142395,10.34
from-country,107107,7.77
has-genre,90415,6.56
has-author,67511,4.90
has-population,47657,3.46
headquarters,34948,2.54
